# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

/Users/pedroconsuegramateo/opt/miniconda3/envs/m2_ih/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
step1 = data.groupby(by=['CustomerID', 'ProductName'], as_index=False)[['Quantity']].sum()
step1

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
step2 = pd.pivot_table(step1, values='Quantity', columns='CustomerID', index='ProductName', fill_value=0)
step2 = step2.transpose()
step2

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:

step3_distance = pdist(X=step2, metric='euclidean')
step3_distribution = squareform(step3_distance)
step3_euclid_dist = pd.DataFrame(step3_distribution,
                           index=step2.index, 
                           columns=step2.index)

step3_euclid_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [7]:
# por si acaso...
#step3_distance = pdist(X=step2, metric='euclidean')
#step3_distribution = squareform(step3_distance)
#step3_euclid_dist = pd.DataFrame(step3_distribution,
#                           index=step2.index, 
#                           columns=step2.index)
#
#step3_euclid_dist

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
def top5ById(id):
    return step3_euclid_dist[id].sort_values(ascending=False).head()



In [9]:
top533 = top5ById(33)
top533 = pd.DataFrame(top533)
top533

,33
CustomerID,
92255,248.594047
97324,247.024290
95017,244.276483
92492,241.642298
91777,240.547293


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
index = list(top533.index)
index

[92255, 97324, 95017, 92492, 91777]

In [11]:
top533_step5 = step1[step1['CustomerID'] == index[0]]
top533_step5 = top533_step5.append(step1[step1['CustomerID'] == index[1]],ignore_index=True)
top533_step5

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/1731635055.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top533_step5 = top533_step5.append(step1[step1['CustomerID'] == index[1]],ignore_index=True)


,CustomerID,ProductName,Quantity
0,92255,"Appetizer - Mini Egg Roll, Shrimp",24
1,92255,Appetizer - Sausage Rolls,24
2,92255,Apricots - Dried,24
3,92255,Arizona - Green Tea,24
4,92255,Bagel - Plain,24
...,...,...,...
147,97324,Wine - Redchard Merritt,25
148,97324,Wine - Vineland Estate Semi - Dry,25
149,97324,"Wine - White, Colubia Cresh",25
150,97324,Wonton Wrappers,25


In [31]:
top533_step5 = step1[step1['CustomerID'] == index[0]]
for i in index:
    if i != index[0]:
        pass
    top533_step5 = top533_step5.append(step1[step1['CustomerID'] == i],ignore_index=True)

top533_step5['CustomerID'].unique()

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/3581993946.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top533_step5 = top533_step5.append(step1[step1['CustomerID'] == i],ignore_index=True)


array([92255, 97324, 95017, 92492, 91777])

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [29]:
agg = top533_step5.groupby(by='ProductName')['Quantity'].sum()
agg = pd.DataFrame(agg)
agg

,Quantity
ProductName,
"Appetizer - Mini Egg Roll, Shrimp",72
Appetizer - Mushroom Tart,49
Appetizer - Sausage Rolls,48
Apricots - Dried,48
Apricots - Halves,24
...,...
"Wine - White, Schroder And Schyl",96
Wonton Wrappers,25
Yoghurt Tubes,24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [20]:
# Chosen id costumer: 33
customer_33 = step1[step1['CustomerID'] == 33]
step7 = agg.merge(customer_33, how='right', on='ProductName')
step7 = step7.fillna(value=0)
list(step7[step7['Quantity_x'] == 0.0].sort_values(by='Quantity_y', ascending=False)['ProductName'].head())

['Veal - Inside, Choice',
 'Grouper - Fresh',
 'Lettuce - Spring Mix',
 'Assorted Desserts',
 'Pate - Cognac']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [34]:
# Functions

def getIds():
    return list(step1['CustomerID'].unique())

def top5ById(id_):
    return step3_euclid_dist[id_].sort_values(ascending=False).head()

def top5Indexes(df):
    return list(df.index)

def top5Customers(step1, indexes):
    top5 = step1[step1['CustomerID'] == indexes[0]]
    for i in indexes:
        if i != indexes[0]:
            pass
        top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)

    return top5

def aggregationPurchaseRecords(top5Customers):
    agg = top5Customers.groupby(by='ProductName')['Quantity'].sum()
    agg = pd.DataFrame(agg)
    return agg

def recommendedProducts(step1, id_, agg):
    customer = step1[step1['CustomerID'] == id_]
    rec = agg.merge(customer, how='right', on='ProductName')
    rec = rec.fillna(value=0)
    return list(rec[rec['Quantity_x'] == 0.0].sort_values(by='Quantity_y', ascending=False)['ProductName'].head())

In [43]:
customers = getIds()
perRecs = {}
for i in customers:
    
    top5 = top5ById(i)
    
    indexes = top5Indexes(top5)
    
    top5Cust = top5Customers(step1, indexes)
    
    
    agg = aggregationPurchaseRecords(top5Cust)
    
    recs = recommendedProducts(step1, i, agg)
    
    perRecs[i] = recs
    
perRecs

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

...

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

{33: ['Veal - Inside, Choice',
  'Grouper - Fresh',
  'Lettuce - Spring Mix',
  'Assorted Desserts',
  'Pate - Cognac'],
 200: ['Lamb - Ground',
  'Loquat',
  'Wiberg Super Cure',
  'Water - Spring Water 500ml',
  'Truffle Cups - Brown'],
 264: ['Apricots Fresh',
  'Oil - Shortening,liqud, Fry',
  'Wine - Sogrape Mateus Rose',
  'Wine - Chablis 2003 Champs',
  'Turkey - Oven Roast Breast'],
 356: ['Mayonnaise - Individual Pkg',
  'Tomatoes Tear Drop',
  'Sobe - Tropical Energy',
  'Scallops - 10/20',
  'Baking Powder'],
 412: ['Flavouring - Orange',
  'Sauce - Demi Glace',
  'Pernod',
  'Beef - Rib Eye Aaa',
  'Lettuce - Frisee'],
 464: ['Vaccum Bag 10x13',
  'Kellogs All Bran Bars',
  'Apricots Fresh',
  'Rambutan',
  'Wine - Sogrape Mateus Rose'],
 477: ['Wine - Charddonnay Errazuriz',
  'Lamb - Ground',
  'Lettuce - Treviso',
  'Beef - Ground, Extra Lean, Fresh',
  'Pasta - Angel Hair'],
 639: ['Squid - Tubes / Tenticles 10/20',
  'Anchovy Paste - 56 G Tube',
  'Assorted Desserts',


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [48]:
recommendations = pd.DataFrame(perRecs)
recommendations = recommendations.transpose()
recommendations

,0,1,2,3,4
33,"Veal - Inside, Choice",Grouper - Fresh,Lettuce - Spring Mix,Assorted Desserts,Pate - Cognac
200,Lamb - Ground,Loquat,Wiberg Super Cure,Water - Spring Water 500ml,Truffle Cups - Brown
264,Apricots Fresh,"Oil - Shortening,liqud, Fry",Wine - Sogrape Mateus Rose,Wine - Chablis 2003 Champs,Turkey - Oven Roast Breast
356,Mayonnaise - Individual Pkg,Tomatoes Tear Drop,Sobe - Tropical Energy,Scallops - 10/20,Baking Powder
412,Flavouring - Orange,Sauce - Demi Glace,Pernod,Beef - Rib Eye Aaa,Lettuce - Frisee
...,...,...,...,...,...
97928,Bread Crumbs - Panko,Tea - Jasmin Green,Bay Leaf,Puree - Mocha,Pate - Cognac
98069,Mustard Prepared,Hot Chocolate - Individual,Loquat,Nantuket Peach Orange,Olives - Kalamata
98159,"Wine - White, Mosel Gold",Cheese - Parmesan Cubes,Papayas,Squid - Tubes / Tenticles 10/20,Tray - 16in Rnd Blk
98185,Bagel - Plain,Whmis - Spray Bottle Trigger,"Salsify, Organic","Pasta - Penne, Rigate, Dry",Milk - 2%


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [49]:
corr_distance = pdist(X=step2, metric='correlation')
corr_distribution = squareform(corr_distance)
corr_dist = pd.DataFrame(corr_distribution,
                           index=step2.index, 
                           columns=step2.index)

corr_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,1.025211,0.887943,0.959400,0.972579,0.971542,1.051663,1.008305,1.003146,0.990052,...,1.026814,1.064510,1.026400,1.083910,1.000520,1.084359,1.027589,1.055680,1.035068,1.032223
200,1.025211,0.000000,1.001067,1.005715,1.073948,1.075028,0.926265,1.080738,1.003482,0.978399,...,0.981300,1.015439,0.930933,0.964814,1.030125,1.063924,0.870938,0.998990,1.064954,1.046097
264,0.887943,1.001067,0.000000,1.009730,0.947144,1.039546,0.987110,1.079281,0.991922,1.039807,...,0.965398,1.037471,1.001002,1.003357,0.931461,0.903754,0.953715,0.986098,0.938976,1.037179
356,0.959400,1.005715,1.009730,0.000000,1.040222,1.041743,0.992298,0.969268,1.071740,1.056114,...,1.003586,1.054821,1.005365,0.991127,1.023312,1.015781,1.065232,0.991223,0.974719,0.997480
412,0.972579,1.073948,0.947144,1.040222,0.000000,1.009738,0.979762,0.951334,1.024509,0.969367,...,1.048723,0.888568,1.032385,1.019126,0.981172,0.999745,1.077885,0.990375,0.976296,0.941690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,1.084359,1.063924,0.903754,1.015781,0.999745,0.968645,0.981220,1.002805,0.998928,1.029966,...,1.021932,0.981406,1.050846,0.960573,0.870999,0.000000,0.902651,0.964935,1.032316,1.070734
98069,1.027589,0.870938,0.953715,1.065232,1.077885,1.065183,1.012509,1.039542,1.018858,0.976137,...,1.043515,1.001031,1.027032,1.063468,0.930410,0.902651,0.000000,0.954242,0.995756,1.004420
98159,1.055680,0.998990,0.986098,0.991223,0.990375,1.003224,0.999040,0.991565,0.959505,0.991348,...,0.979906,0.913048,0.997792,1.044248,0.974952,0.964935,0.954242,0.000000,0.982682,1.017836


In [52]:
# New function for correlation distance
def top5ByIdCorr(id_):
    return corr_dist[id_].sort_values(ascending=False).head()

In [59]:
customersCorr = getIds()
perRecsCorr = {}
for i in customers:
    
    top5 = top5ByIdCorr(i)
   
    indexes = top5Indexes(top5)
    
    top5Cust = top5Customers(step1, indexes)
    
    
    agg = aggregationPurchaseRecords(top5Cust)
    
    recs = recommendedProducts(step1, i, agg)

    perRecsCorr[i] = recs
    
perRecsCorr

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

...

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5 = top5.append(step1[step1['CustomerID'] == i],ignore_index=True)
/var/folders/n3/xb6jll3d49q147jw5_13y6zm0000gn/T/ipykernel_11338/510048971.py:17: FutureWarning: The frame.append method is deprecated and 

{33: ['Salsify, Organic',
  'Lettuce - Spring Mix',
  'Veal - Inside, Choice',
  'Grouper - Fresh',
  'Assorted Desserts'],
 200: ['Lamb - Ground',
  'Olives - Kalamata',
  'Remy Red',
  'Tahini Paste',
  'Scallop - St. Jaques'],
 264: ['Mushrooms - Black, Dried',
  'Guinea Fowl',
  'Chocolate - Dark',
  'Snapple Lemon Tea',
  'Pastry - Butterscotch Baked'],
 356: ['Tomatoes Tear Drop',
  'Mayonnaise - Individual Pkg',
  'Scallops - 10/20',
  'Cinnamon Buns Sticky',
  'Sobe - Tropical Energy'],
 412: ['Ecolab - Lime - A - Way 4/4 L',
  'Pernod',
  'Sauce - Demi Glace',
  'Spinach - Baby',
  'Flavouring - Orange'],
 464: ['Vaccum Bag 10x13',
  'Phyllo Dough',
  'Kellogs All Bran Bars',
  'Gloves - Goldtouch Disposable',
  'Banana Turning'],
 477: ['Bacardi Breezer - Tropical',
  'Cheese - Mix',
  'Wine - Charddonnay Errazuriz',
  'Soup - Campbells Bean Medley',
  'Lettuce - Treviso'],
 639: ['Wine - Toasted Head',
  'Juice - Apple Cider',
  'Squid - Tubes / Tenticles 10/20',
  'Appetize

In [61]:
corrRecommendations = pd.DataFrame(perRecsCorr)
corrRecommendations = corrRecommendations.transpose()
corrRecommendations

,0,1,2,3,4
33,"Salsify, Organic",Lettuce - Spring Mix,"Veal - Inside, Choice",Grouper - Fresh,Assorted Desserts
200,Lamb - Ground,Olives - Kalamata,Remy Red,Tahini Paste,Scallop - St. Jaques
264,"Mushrooms - Black, Dried",Guinea Fowl,Chocolate - Dark,Snapple Lemon Tea,Pastry - Butterscotch Baked
356,Tomatoes Tear Drop,Mayonnaise - Individual Pkg,Scallops - 10/20,Cinnamon Buns Sticky,Sobe - Tropical Energy
412,Ecolab - Lime - A - Way 4/4 L,Pernod,Sauce - Demi Glace,Spinach - Baby,Flavouring - Orange
...,...,...,...,...,...
97928,Apricots - Halves,Tea - Jasmin Green,Bouq All Italian - Primerba,Cookie Dough - Double,Bread Crumbs - Panko
98069,Mustard Prepared,Hot Chocolate - Individual,"Chocolate - Semi Sweet, Calets",Apricots - Halves,Snapple Lemon Tea
98159,Beef - Ground Medium,"Wine - White, Mosel Gold",Papayas,Campari,Cheese - Parmesan Cubes
98185,Scallops - Live In Shell,Bagel - Plain,Mussels - Cultivated,Whmis - Spray Bottle Trigger,"Salsify, Organic"
